In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

from code_source.dataframe_prep import get_cols_names
from code_source import data_prep
from code_source import metrics
from code_source import learning

# dataframe preparation

In [2]:
data=pd.read_csv('data/Dataset/Dataset/Training/Features_Variant_1.csv', header=None).drop_duplicates()
data=get_cols_names(data)
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative

# data preparation

In [3]:
#'page_cat' to dummy because it is a category
#data=pd.concat([data,pd.get_dummies(data['page_cat'], prefix='page_cat')], axis=1)
#data=data.drop(columns=['page_cat'])

In [4]:
#data shuffling
data=shuffle(data).reset_index(drop=True)

# learning

In [5]:
#set algo params:
lam=1e-2
gamma=0 #now we needn't it
terms_num=10000
max_iter=1

# Cross-val

In [6]:
data.iloc[:,:-1], mean, std =data_prep.normalize(data.iloc[:,:-1])

res=pd.DataFrame()

parts=range(1,6)
idxs=[round(data.shape[0]/5)*(i-1) for i in range(1,6)]
idxs.append(data.shape[0])

for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])

    #features normalization (xi-mean)/std_err:
    #train_part.iloc[:,:-1], mean, std=normalize(train_part.iloc[:,:-1])#fit and transform train
    #test_part.iloc[:,:-1]=normalize_with_params(test_part.iloc[:,:-1], mean, std) #transform test
    
    params=learning.gradient_descend(train_part, lam, gamma, terms_num, max_iter) #model    
    test_pred=learning.get_prediction(params[:-1],params[-1],test_part.iloc[:,:-1])
    RMSE=metrics.custom_RMSE(test_part['target'], test_pred, gamma, params[:-1])
    r2=metrics.custom_R2(test_part['target'], test_pred)
    params=np.insert(params, 0 , [RMSE,r2])
    res[part]=params

[18803  8857 24695 ... 11983  2484 23120]
<class 'numpy.ndarray'>
[ 2691  6331  3407 ...   825  7804 31238]
<class 'numpy.ndarray'>
[19052  7453  1101 ... 15506  7584 21164]
<class 'numpy.ndarray'>
[ 8322 16507 28786 ... 11341  7972  6180]
<class 'numpy.ndarray'>
Iterations number is:1
The maximum number of iterations was reached.
RMSE is:36.81781513063726
R2 is:-0.039378204470921885
[22731   513  6718 ...   937  2516  6403]
<class 'numpy.ndarray'>
[21037 17983 30359 ...  6853  6374  7520]
<class 'numpy.ndarray'>
[26324 29789  6434 ... 13341 32126 12528]
<class 'numpy.ndarray'>
[31103 10662 16142 ... 17567 28471 11939]
<class 'numpy.ndarray'>
Iterations number is:1
The maximum number of iterations was reached.
RMSE is:36.136964214861514
R2 is:-0.04111717232470169
[25471 11070 13859 ... 13954 31959  7443]
<class 'numpy.ndarray'>
[15897 25533 13534 ... 29877 10690  2877]
<class 'numpy.ndarray'>
[18006 17369   183 ... 24338  5050 30836]
<class 'numpy.ndarray'>
[19934  3072 12867 ... 18053

In [7]:
text_idxs=np.array(['w'+str(i) for i in range(1,res.shape[0]-2)])
text_idxs=np.append(text_idxs,'w0')
text_idxs=np.insert(text_idxs, 0, ['RMSE', 'r2'])
res.index=text_idxs

res['mean']=np.mean(res, axis=1)
res['std']=np.std(res.iloc[:,:-1], axis=1)

In [8]:
res

,1,2,3,4,5,mean,std
RMS,33.694860,36.506905,37.509125,37.258516,35.965994,36.187080,1.360743
r2,-0.047789,-0.038695,-0.038091,-0.038637,-0.041692,-0.040981,0.003631
w1,0.000240,-0.001887,-0.000214,-0.002037,-0.001203,-0.001020,0.000901
w2,-0.001329,0.005172,0.001302,0.003731,0.007901,0.003355,0.003171
w3,0.001389,0.001397,0.006254,-0.001006,0.001241,0.001855,0.002381
w4,-0.000393,-0.004037,-0.005299,-0.000029,-0.001325,-0.002216,0.002084
w5,0.004368,-0.002059,-0.000146,-0.001466,-0.002001,-0.000261,0.002414
w6,0.002498,0.000727,0.001500,-0.004509,-0.000356,-0.000028,0.002428
w7,0.003506,0.001971,0.005221,-0.001840,0.003631,0.002498,0.002400
w8,0.002635,0.000520,0.004820,-0.001360,0.003150,0.001953,0.002151
